# SARSA Frozen Lake
Implementation of a SARSA agent to learn policies in the Frozen Lake environment from OpenAI gym.

In [1]:
# import dependency
import gym # OpenAI Game Environment
import gym.envs.toy_text # Customized Map
import numpy as np
from tqdm import trange # Processing Bar
%matplotlib inline

## Problem - Frozen Lake

[Frozen Lake](https://gym.openai.com/envs/FrozenLake-v0/) is an environment where an agent is able to move a character in a grid world. Starting from the state *S*, the agent aims to move the character to the goal state *G* for a reward of 1. Although the agent can pick one of four possible actions at each state including *left*, *down*, *right*, *up*, it only succeeds $\frac{1}{3}$ of the times due to the slippery frozen state *F*. The agent is likely to move to any other directions for the remaining $\frac{2}{3}$ times evenly. Additionally, stepping in a hole state *H* will lead to a bad ending with a reward of 0.

+ S: Start State
+ G: Goal State
+ F: Frozen Surface
+ H: Hole State

![Frozen Lake](img/frozen_lake_0.png)

In [4]:
# initialization
amap='SFFFHFFFFFFFFFFG'
grid_shape = np.int(np.sqrt(len(amap)))
custom_map = np.array(list(amap)).reshape(grid_shape, grid_shape)
env = gym.envs.toy_text.frozen_lake.FrozenLakeEnv(desc=custom_map, is_slippery=True).unwrapped
# env = gym.make('FrozenLake-v0')
n_states, n_actions = env.observation_space.n, env.action_space.n
print('{} states'.format(n_states))
print('{} actions'.format(n_actions))
env.render()    

16 states
4 actions

SFFF
HFFF
FFFF
FFFG


In [5]:
# take a look
done = False
env.reset()
while not done:
    # randomly pick an action
    action = np.random.randint(n_actions)
    # get feedback from the environment
    obvervation, reward, done, info = env.step(action)
    # show the environment
    env.render()

  (Right)
SFFF
HFFF
FFFF
FFFG


## State Action Reward State Action (SARSA)

[State Action Reward State Action (SARSA)](https://en.wikipedia.org/wiki/State%E2%80%93action%E2%80%93reward%E2%80%93state%E2%80%93action) is a classic method learning a Markov Decision Process (MDP) policy to solve problems in the field of reinforcement learning. As indicated by the name SARSA, it updates the $Q(s_{t}, a_{t})$, according to the current state $s_{t}$, the action choose $a_{t}$, the reward $r_{t}$ due to this action, the new state $s_{t+1}$ after taking this action, and the action $a_{t+1}$ picked for this new state.
Given that, the Q-value table can be updated by:

$$Q(s_{t}, a_{t}) \leftarrow Q(s_{t}, a_{t}) + \alpha[r_{t} + \gamma Q(s_{t+1}, a_{t+1}) - Q(s_{t}, a_{t})]$$

where, $\alpha$ stands for the learning rate and $\gamma$ represents the discount factor. It can be seen in the definition that the SARSA method aims to update the policy through interactions with the environment, so it belongs to the on-policy learning algorithm family.

### Training

In [6]:
# initialize the agent’s Q-table to zeros
def init_q(s, a):
    """
    s: number of states
    a: number of actions
    """
    return np.zeros((s, a))

# epsilon-greedy exploration strategy
def epsilon_greedy(Q, epsilon, n_actions, s):
    """
    Q: Q Table
    epsilon: exploration parameter
    n_actions: number of actions
    s: state
    """
    # selects a random action with probability epsilon
    if np.random.random() <= epsilon:
        return np.random.randint(n_actions)
    else:
        return np.argmax(Q[s, :])
    
# SARSA Process
def sarsa(alpha, gamma, epsilon, n_episodes):
    """
    alpha: learning rate
    gamma: exploration parameter
    n_episodes: number of episodes
    """
    # initialize Q table
    Q = init_q(n_states, n_actions)
    t = trange(n_episodes)
    reward_array = np.zeros(n_episodes)
    for i in t:
        # initial state
        s = env.reset()
        # initial action
        a = epsilon_greedy(Q, epsilon, n_actions, s)
        done = False
        while not done:
            s_, reward, done, _ = env.step(a)
            a_ = epsilon_greedy(Q, epsilon, n_actions, s_)
            # update Q table
            Q[s, a] += alpha * (reward + (gamma * Q[s_, a_]) - Q[s, a])
            if done:
#                 t.set_description('Episode {} Reward {}'.format(i + 1, reward))
#                 t.refresh()
                reward_array[i] = reward
                break
            s, a = s_, a_
    env.close()
    return Q, reward_array 

In [7]:
# experiment settings
alpha = 0.25 # learning rate
gamma = 1.0 # discount factor
epsilon=0.29 # exploration parameter
n_episodes = 14697  # number of training episodes
np.random.seed(741684) 

In [10]:
# training
Q, reward = sarsa(alpha, gamma, epsilon, n_episodes)

Episode 14697 Reward 1.0: 100%|██████████| 14697/14697 [01:06<00:00, 220.96it/s]


In [11]:
# show Q table
Q

array([[0.36056764, 0.54765592, 0.53477343, 0.75206226],
       [0.68993437, 0.6489726 , 0.72299097, 0.82219617],
       [0.869973  , 0.88669754, 0.93351916, 0.82829123],
       [0.93381796, 0.93253269, 0.94729486, 0.91578574],
       [0.        , 0.        , 0.        , 0.        ],
       [0.26542696, 0.22786288, 0.85165137, 0.73450378],
       [0.87363453, 0.89477981, 0.92076252, 0.89000001],
       [0.93498558, 0.96450615, 0.95163416, 0.91824267],
       [0.61137549, 0.84120554, 0.4918512 , 0.62596831],
       [0.66558452, 0.89029202, 0.82599748, 0.74539858],
       [0.88804912, 0.94793042, 0.91143964, 0.91659273],
       [0.95812485, 0.98552116, 0.96425939, 0.95837349],
       [0.85681271, 0.86301515, 0.85280369, 0.8206385 ],
       [0.88022332, 0.86549787, 0.9247272 , 0.77877382],
       [0.93910151, 0.97785198, 0.94577155, 0.93788063],
       [0.        , 0.        , 0.        , 0.        ]])

### Testing

In [19]:
# trained SARSA agent in Frozen Lake
done = False
s = env.reset()
# env.render()
actions = []
while not done:
    # pick an action
    a = np.argmax(Q[s])
    actions.append(a) 
    # get feedback from the environment
    s_, _, done, _ = env.step(a)
    # show the environment
#     env.render()
    s = s_
actions

[3, 3, 3, 3, 2, 2, 1, 1, 1, 1, 2, 2, 1, 2, 2, 1, 2, 2, 1, 2, 1]